# QubiC DAC to Oscilloscope Test

In [ ]:
import sys
sys.path.append('/home/xilinx/.local/lib/python3.9/site-packages')

In [3]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import numpy as np
import matplotlib.pyplot as plt 

In [4]:
fpga_config = FPGAConfig() 
# fpga_config = FPGAConfig(**{
#     'fpga_clk_period': 2e-9,  # 500 MHz = 2ns clock
#     'alu_instr_clks': 5,
#     'jump_cond_clks': 5,
#     'jump_fproc_clks': 5,
#     'pulse_regwrite_clks': 3
# })

In [5]:
channel_configs = load_channel_configs('channel_config.json') 

In [6]:
runner = rc.CircuitRunnerClient(ip='127.0.0.1', port=9095) 

In [13]:
circuit = []

for q in range(8):
    for drv in ['qdrv']:
        circuit.append({
            'name': 'pulse',
            'phase': 0,
            'freq': 150000000,
            'amp': 1,
            'twidth': 0.5e-6,
            'env': {
                'env_func': 'cos_edge_square',
                'paradict': {
                    'ramp_fraction': 0.5
                }
            },
            'dest': f'Q{q}.{drv}'
        })


In [14]:
compiled_prog = tc.run_compile_stage(circuit, fpga_config = fpga_config, qchip = None, compiler_flags={'resolve_gates': False})
compiled_prog

CompiledProgram(program={('Q3.qdrv', 'Q3.rdrv', 'Q3.rdlo'): [{'op': 'phase_reset'}, {'op': 'pulse', 'freq': 150000000, 'phase': 0, 'amp': 1, 'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.5, 'twidth': 5e-07}}, 'start_time': 5, 'dest': 'Q3.qdrv'}, {'op': 'done_stb'}], ('Q7.qdrv', 'Q7.rdrv', 'Q7.rdlo'): [{'op': 'phase_reset'}, {'op': 'pulse', 'freq': 150000000, 'phase': 0, 'amp': 1, 'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.5, 'twidth': 5e-07}}, 'start_time': 5, 'dest': 'Q7.qdrv'}, {'op': 'done_stb'}], ('Q5.qdrv', 'Q5.rdrv', 'Q5.rdlo'): [{'op': 'phase_reset'}, {'op': 'pulse', 'freq': 150000000, 'phase': 0, 'amp': 1, 'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.5, 'twidth': 5e-07}}, 'start_time': 5, 'dest': 'Q5.qdrv'}, {'op': 'done_stb'}], ('Q6.qdrv', 'Q6.rdrv', 'Q6.rdlo'): [{'op': 'phase_reset'}, {'op': 'pulse', 'freq': 150000000, 'phase': 0, 'amp': 1, 'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_

In [15]:
asm_prog = tc.run_assemble_stage(compiled_prog, channel_configs) 

In [16]:
runner.run_circuit_batch([asm_prog], n_total_shots=1000000)

{'Q5.rdlo': array([[[2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         ...,
         [0.00000000e+00+0.00000000e+00j],
         [0.00000000e+00+0.00000000e+00j],
         [0.00000000e+00+0.00000000e+00j]]]),
 'Q6.rdlo': array([[[2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         ...,
         [0.00000000e+00+0.00000000e+00j],
         [0.00000000e+00+0.00000000e+00j],
         [0.00000000e+00+0.00000000e+00j]]]),
 'Q4.rdlo': array([[[2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         ...,
         [0.00000000e+00+0.00000000e+00j],
         [0.00000000e+00+0.00000000e+00j],
         [0.00000000e+00+0.00000000e+00j]]]),
 'Q0.rdlo': array([[[2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
         [2.14741811e+09+2.14741811e+09j],
  